In [ ]:
import pandas as pd
from keras.layers import *
from keras.models import Model
from scripts.AnswerExtraction.data_helper import *
from scripts.AnswerExtraction.evaluation import *
from numpy.random import seed
from tensorflow import set_random_seed
from keras.optimizers import adam
import re, string
import numpy as np, pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from scripts.AnswerExtraction import expand_contractions as expand

seed(1)
set_random_seed(2)

#########################################################former_QuestionPattern.py#############################################################
import os
import pandas as pd
import spacy
import re
from spacy.matcher import Matcher
from sklearn.metrics import precision_recall_fscore_support
from spacy.attrs import LOWER, POS, ENT_TYPE, LEMMA
from spacy.tokens import Doc
import numpy

rec_target_noun = ["project","api","tutorial","library","example","book","resource","elm","python","clojure",
                    "way","practice","approach","strategy","technique","explanation","choice", "trick","tip",
                    "alternative","recommendation","opinion", "boilerplate","service","framework","project","package",
                    "suggestion","linter","procedure"]
action_verbs = ["have", "know","use","try"]
rec_verbs = ["point","recommend","suggest","advise","tell"]
rec_positive_adjective = ["good","better","best","right","optimal","ideal","common","established","popular","great",
                          "nice","nicer","elegant","efficient","easy","easier","simple","simpler","simplistic",
                          "clean","cleaner","pro","other","idiomatic","pythonic","clojurian","clojure","clojury","elm",
                          "elmish","generally","accepted","currently","most","normal","ok","appropriate","standard"]
all_caps_noun_regex = "[A-Z]{2,}s?"
vp_pattern = [{"POS": {"NOT_IN": ["VERB","AUX"]}, "OP": "*"},
              {"POS": {"IN": ["VERB","AUX"]}}]
np_pattern = [{"POS": {"NOT_IN": ["NOUN","PROPN"]}, "OP": "*"},
              {"POS": {"IN": ["NOUN","PROPN"]}}]

# P_WHAT_RECADJ_TARGETNOUN
# Question with postitive adjective and noun
# What|Which [VERB_TO_BE] [ADJECTIVE]* <rec_target_noun>
def setup_p_what_recadj_targetnoun(matcher):
    base_pattern = [{"LEMMA": {"IN": ["what","which"]}},
                    {"LOWER": {"IN": ["would","one"]},"OP": "?"},
                    {"LEMMA": "be", "OP": "?"},
                    {"POS": "DET","OP": "?"},
                    {"POS": "ADJ","OP": "?"}]
    what_is_pattern1 = base_pattern + [
                    {"LEMMA": {"IN": rec_target_noun}}] + vp_pattern
    what_is_pattern2 = base_pattern + [
                    {"TEXT": {"REGEX": all_caps_noun_regex}}] + vp_pattern
    matcher.add("P_WHAT_RECADJ_TARGETNOUN",
                on_match=on_match_p_what_recadj_targetnoun,
                patterns=[what_is_pattern1, what_is_pattern2])

def on_match_p_what_recadj_targetnoun(matcher, doc, id, matches):
    #print('P_WHAT_RECADJ_TARGETNOUN Matched!', doc.text)
    pass


# P_COULD_ANYONE
# Query about the existence of a person with knowledge
# Could|Can anyone|someone rec_verb [me|us] [to] [an] <adjective>* <rec_target_noun>
def setup_p_could_anyone(matcher):
    could_pattern = [{"LOWER": {"IN": ["can", "could"]}},
                     {"LOWER": {"IN": ["someone", "anyone"]}},
                     {"LOWER": {"IN": rec_verbs}}]
    matcher.add("P_COULD_ANYONE", on_match_p_could_anyone, could_pattern)

def on_match_p_could_anyone(matcher, doc, id, matches):
    #print('P_COULD_ANYONE Matched!', doc.text)
    pass


# P_ARE_THERE
# Query about the existence an artifact
# Is|Are there any|a|an|some <rec_positive_adjective> <rec_target_noun>
def setup_p_are_there(matcher):
    at_pattern_base = [{"LOWER": {"IN": ["are","is"]}},
                     {"LOWER": "there"},
                     {"LOWER": {"IN": ["any", "a", "some", "an"]}},
                     {"LOWER": {"IN": rec_positive_adjective}, "OP": "+"}]
    # at_pattern1 = at_pattern_base + [{"LEMMA": {"IN": rec_target_noun}}] + vp_pattern
    # at_pattern2 = at_pattern_base + [{"TEXT": {"REGEX": all_caps_noun_regex}}] + vp_pattern
    matcher.add("P_ARE_THERE",
                on_match = on_match_p_are_there,
                patterns = [at_pattern_base])

def on_match_p_are_there(matcher, doc, id, matches):
    #print('P_ARE_THERE Matched!', doc.text)
    pass


# P_IS_IT
def setup_p_is_it(matcher):
    is_it_pattern = [{"LOWER": "is"},
                     {"LOWER": "it"},
                     {"LOWER": {"IN": rec_positive_adjective}, "OP": "+"}]
    matcher.add("P_IS_IT",
                on_match = on_match_p_is_it,
                patterns = [is_it_pattern])


def on_match_p_is_it(matcher, doc, id, matches):
    pass


# P_DOES_ANYONE
# Query about someone having specific knowledge
# [Do|Does] we|anyone|someone <action_verbs> [...] <rec_target_noun>
def setup_p_does_anyone(matcher):
    does_pattern = [{"LOWER": {"IN": ["does", "do"]}, "OP": "?"},
                    {"LOWER": {"IN": ["we", "anyone", "someone"]}},
                    {"LEMMA": {"IN": action_verbs}},
                    {"LEMMA": {"NOT_IN": rec_target_noun}, "OP": "*"},
                    {"LEMMA": {"IN": rec_target_noun}, "OP": "+"}]
    matcher.add("P_DOES_ANYONE", on_match_p_does_anyone, does_pattern)

def on_match_p_does_anyone(matcher, doc, id, matches):
    #print('P_DOES_ANYONE Matched!', doc.text)
    pass


# P_HAS_ANYONE
# Query about someone having specific knowledge
# Has|Have anyone|someone|you <action_verbs> <noun_phrase>
def setup_p_has_anyone(matcher):
    has_pattern = [{"LOWER": {"IN": ["has", "have"]}},
                    {"LOWER": {"IN": ["you", "anyone", "someone"]}},
                    {"LEMMA": {"IN": action_verbs}}] + \
                    np_pattern
    matcher.add("P_DOES_ANYONE", on_match_p_has_anyone, has_pattern)

def on_match_p_has_anyone(matcher, doc, id, matches):
    #print('P_HAS_ANYONE Matched!', doc.text)
    pass


# P_ANY
# Question starting with any and followed by a rec target
# Any <rec_target_noun> <noun_phrase>
def setup_p_any(matcher):
    base_pattern = [{"LOWER": "any"},
                    {"LOWER": {"IN": rec_positive_adjective}, "OP": "+"}]
    vp_narrow_pattern = [{"POS": {"NOT_IN": ["VERB"]}, "OP": "*"},
                         {"POS": "VERB"}]
    #not_i_pattern = [{"TEXT": {"NOT_IN": ["I"]}}]
    any_pattern1 = base_pattern + [{"LEMMA": {"IN": rec_target_noun}}] + vp_narrow_pattern
    any_pattern2 = base_pattern + [{"TEXT": {"REGEX": all_caps_noun_regex}}] + vp_narrow_pattern
    any_pattern3 = [{"LOWER": "any"}, {"LEMMA": {"IN": ["recommend","recommendation","advice","suggestion","alternative","direction"]}}]
    matcher.add("P_ANY", on_match=on_match_p_any, patterns=[any_pattern1, any_pattern2, any_pattern3])

def on_match_p_any(matcher, doc, id, matches):
    #print('P_ANY Matched!', doc.text)
    pass


# P_WHERE_FIND
# Question about the location of a rec item
# Where can|could I|someone|anyone find <rec_target_noun>
def setup_p_where(matcher):
    end_pattern =  [{"LOWER": "find"},
                    {"LEMMA": {"NOT_IN": rec_target_noun}, "OP": "*"},
                    {"LEMMA": {"IN": rec_target_noun}, "OP": "+"}]
    where_pattern1 = [{"LOWER": "where"},
                    {"LOWER": {"IN": ["can","could"]}},
                    {"LOWER": {"IN": ["i", "anyone", "someone"]}},
                    {"LOWER": "find"}] + end_pattern
    where_pattern2 = [{"LOWER": "where"},
                    {"LOWER": "i"},
                    {"LOWER": {"IN": ["can","could"]}}] + end_pattern
    matcher.add("P_ANY", on_match=on_match_p_where, patterns=[where_pattern1, where_pattern2])

def on_match_p_where(matcher, doc, id, matches):
    #print('P_WHERE Matched!', doc.text)
    pass


# P_SHOULD_I_OR
def setup_p_should_i_or(matcher):
    or_pattern1 = [{"LOWER": {"IN": ["should"]}},
                    {"LOWER": {"IN": ["i"]}}] + \
                    np_pattern + \
                  [{"LOWER": {"NOT_IN": ["or"]}, "OP":"*"},
                   {"LOWER": {"IN": ["or"]}}]
    or_pattern2 = [{"LOWER": {"IN": ["or"]}},
                   {"LOWER": {"IN": ["should"]}},
                   {"LOWER": {"IN": ["i"]}}]

    matcher.add("P_DOES_ANYONE", on_match=on_match_p_should_i_or, patterns=[or_pattern1, or_pattern2])

def on_match_p_should_i_or(matcher, doc, id, matches):
    pass


# belong to a single speaker
def filter_potential_questions(all_uter):
    mask = []
    last_thread = -1
    same_speaker = "None"
    for index, row in all_uter.iterrows():
        if row["thread"] != last_thread:
            last_thread = row["thread"]
            same_speaker = row["speaker"]
            mask.append(True)
        elif row["speaker"] == same_speaker:
            mask.append(True)
        else:
            same_speaker = "None"
            mask.append(False)
    return all_uter[mask]


def remove_punct_from_doc(doc):
    indexes = []
    for index, token in enumerate(doc):
        if (token.pos_  in ('PUNCT', 'NUM', 'SYM')):
            indexes.append(index)
    np_array = doc.to_array([LOWER, POS, ENT_TYPE, LEMMA])
    np_array = numpy.delete(np_array, indexes, axis = 0)
    doc2 = Doc(doc.vocab, words=[t.text for i, t in enumerate(doc) if i not in indexes])
    doc2.from_array([LOWER, POS, ENT_TYPE, LEMMA], np_array)
    return doc2


def print_err_diagnostics(data, sent_matches, index):
    if ("is_rec_question" in data.columns):
        if sent_matches:
            if (data["is_rec_question"][index] == 0):
                print("False Positive,\"" + str(data["thread"][index]) + ": " + data["message"][index])
                #print("False Positive,\"" + data["message"][index] + "\"")
        else:
            if (data["is_rec_question"][index] == 1):
                print("False Negative,\"" + data["message"][index] + "\"")


def process_matches(q_data, matcher):
    uter_rec_q = []
    for index, row in q_data.iterrows():
        uter = row['message']
        thread = row['thread']

        if (not isinstance(uter, str)) or '```' in uter:
            uter_rec_q.append(0)
            continue

        sent_matches = False
        matches = []
        for sentence in nlp(uter).sents:
            sent_nlp = nlp(sentence.text)
            doc = remove_punct_from_doc(sent_nlp)
            matches = matcher(doc)
            if (len(matches) >= 1):
                sent_matches = True
        print_err_diagnostics(q_data, sent_matches, index)

        if sent_matches:
            uter_rec_q.append(thread)
        else:
            uter_rec_q.append(0)

    return uter_rec_q


def run_patterns_dev_set(matcher):
    data = pd.read_csv('../../data/question/test_set.csv')
    q_data = filter_potential_questions(data)

    #print("Total convos:", len(data["thread"].unique()))
    #print("Total rec questions:", len(data.loc[data['is_rec_question'] == 1]))

    uter_rec_q = process_matches(q_data, matcher)

    assert(len(uter_rec_q) == len(q_data['is_rec_question']))
    precision, recall, fscore, support = \
        precision_recall_fscore_support(q_data['is_rec_question'], [int(u != 0) for u in uter_rec_q], average='binary')

    print('precision: {}'.format(precision))
    print('recall: {}'.format(recall))
    print('fscore: {}'.format(fscore))
    print('support: {}'.format(support))


def run_patterns_test_set_single(chatFileName, matcher):
    data = pd.read_csv(chatFileName)

    q_data = filter_potential_questions(data)
    uter_rec_q = process_matches(q_data, matcher)
    return data.loc[data['thread'].isin(uter_rec_q)]


def run_patterns_test_set(matcher):
    # generating train set with conversations with rec-asking questions
    rec_threads = 0

    outfile = 'outfile.csv'
    if os.path.exists(outfile):
        os.remove(outfile)

    walkdir = "../../data/irc"
    for root, dirnames, filenames in os.walk(walkdir):
        for filename in filenames:
            if not filename.endswith('.csv'):
                continue
            else:
                chatFileName = os.path.join(root, filename)
                print(chatFileName)

                out_df = run_patterns_test_set_single(chatFileName, matcher)
                ths = len(out_df.thread.unique())
                print("Conversations with rec-asking questions:", ths)
                rec_threads = rec_threads + ths

                #write a new csv file containing only the conversations which start with rec-asking question
                with open(outfile, 'a') as fpointer:
                    #local_path = [chatFileName] * len(df.index)
                    #df['FileName'] = local_path
                    out_df['thread'] = out_df['thread'].apply(lambda x: str(x) + '_' + str(root)[11:].replace("/","_"))
                    assert ths == len(out_df.thread.unique())
                    out_df.to_csv(fpointer, header=False, index=False)
    print("Overall conversations with rec-asking questions:", rec_threads)


if __name__ == '__main__':
    nlp = spacy.load("en_core_web_sm")
    matcher = Matcher(nlp.vocab, validate=True)
    setup_p_what_recadj_targetnoun(matcher)
    setup_p_could_anyone(matcher)
    setup_p_are_there(matcher)
    setup_p_any(matcher)
    setup_p_does_anyone(matcher)
    setup_p_has_anyone(matcher)
    setup_p_where(matcher)
    setup_p_should_i_or(matcher)
    setup_p_is_it(matcher)
    #setup_p_how_do_i(matcher)

    run_patterns_dev_set(matcher)
    #run_patterns_test_set(matcher)

##########################################former_data_helper.py##########################################################################
if __name__ == '__main__':
    df_train = pd.read_csv('../../data/answer/train_set.csv')
    df_test = pd.read_csv('../../data/answer/test_set.csv')


    def text_preprocess(message):
        message = str(message)
        message = re.sub('<http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'url',
                         message)  # Replace urls
        message = re.sub('<@[A-Z][a-z]*>', ' username', message)  # Replace user mentions
        message = re.sub(':[^:\s]*(?:::[^:\s]*)*:', 'emoji', message)  # Replace emoji
        message = re.sub('’', '\'', message)  # handle utf encodings
        message = expand.expand_contractions(message)  # Expand contractions
        message = re.sub('```([^`]*)```', 'code', message)  # Replace only multiline code
        message = re.sub('`([^`]*)`', 'code', message)  # Replace any code
        message = message.lower()  # Lower the text
        message = message.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
        return message


    def get_GloVe_embeddings():
        # Load the customized GloVe embeddings trained on SO and put in Data Folder
        embeddings_index = {}
        f = open('../../data/WordEmbeddings/SO_vectors.txt', 'rb')
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        f.close()
        print('Found %s word vectors.' % len(embeddings_index))
        return embeddings_index


    def get_embedding_matrix(embeddings_index, word_index, num_words, embedding_dim, max_features):
        # first create a matrix of zeros, this is our embedding matrix
        embedding_matrix = np.zeros((num_words, embedding_dim))

        # for each word in out tokenizer lets try to find that work in our w2v model
        for word, i in word_index.items():
            if i > max_features:
                continue
            embedding_vector = embeddings_index.get(bytes(word, 'utf-8'))
            if embedding_vector is not None:
                # add word vector to the matrix
                embedding_matrix[i] = embedding_vector
            else:
                # if word doesn't exist, assign a random vector within our embedding range
                embedding_matrix[i] = np.random.uniform(low=0.001, high=0.009, size=(embedding_dim,))
        return embedding_matrix


    def get_conv_len(df):
        conv_lengths = []
        grouped_by_conv = df.groupby(['thread'], sort=False)
        for thread, conv in grouped_by_conv:
            conv_lengths.append(conv.shape[0])
        return conv_lengths


    def create_utterance_embeddings(X, embedding_matrix):
        X_repr = []
        for utterance in X:
            vec = []
            for word in utterance:
                try:
                    vec.append(embedding_matrix[word])
                except KeyError:
                    continue
            X_repr.append(vec)
        X_repr = np.asarray(X_repr)
        return X_repr


    def pad_nested_sequences(sequences, padding_len):
        pad_seq = []
        for seq in sequences:
            x = []
            count = 0
            vec = np.zeros(200)
            for word in seq:
                if count < padding_len:
                    x.append(word)
                    count += 1
            if len(x) < padding_len:
                for i in range(padding_len - len(x)):
                    x.append(vec)
            pad_seq.append(x)
        pad_seq = np.asarray(pad_seq)
        return pad_seq


    def create_sets(df, tokenizer, embedding_matrix):
        # Data pre-processing
        df['text'] = df['message'].apply(text_preprocess)
        # this takes our sentences and replaces each word with an integer
        X = tokenizer.texts_to_sequences(df['text'].values)
        # Create embeddings
        X0 = create_utterance_embeddings(X, embedding_matrix)
        X1 = pad_nested_sequences(X0, 20)
        Y1 = df['is_rec'].values
        X2 = []
        Y2 = []
        count = 0
        # get all conversation lengths
        conv_lengths = get_conv_len(df)
        sample_weights = []
        # Creating joint QA representations
        for conv in conv_lengths:
            x_temp = []
            y_temp = []
            weights = []
            for length in range(conv):
                if length == 0:
                    ques_repr = X1[count]
                else:
                    candidate_ans_repr = X1[count]
                    concat_QA = np.concatenate((ques_repr, candidate_ans_repr), axis=0)
                    x_temp.append(concat_QA)

                    if Y1[count] == 2:
                        y_temp.append(1)
                        weights.append(1.0)
                    else:
                        y_temp.append(0)
                        weights.append(1.0)
                count += 1
            X2.append(x_temp)
            Y2.append(y_temp)
            sample_weights.append(weights)

        X2 = np.asarray(X2)
        # Padding data for LSTM
        X2 = pad_sequences(X2, maxlen=20, padding="post", truncating="post", dtype='float32', value=0.)
        Y2 = pad_sequences(Y2, maxlen=20, padding="post", truncating="post")
        sample_weights = pad_sequences(sample_weights, maxlen=20, padding="post", truncating="post", value=0.0)
        Y2 = np.asarray(Y2)
        Y2 = Y2.reshape(Y2.shape[0], Y2.shape[1], 1)
        return X2, Y2, sample_weights


    def prep_data(df_train, df_test, embedding_dim):
        df = pd.concat([df_train, df_test])
        # Data pre-processing
        df['text'] = df['message'].apply(text_preprocess)
        # Build vocabulary
        max_features = 30000  # number of words we care about
        tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
        tokenizer.fit_on_texts(df['text'].values)
        word_index = tokenizer.word_index
        print('Found %s unique tokens in vocab' % len(word_index))
        num_words = min(max_features, len(word_index)) + 1
        # get all GloVe embeddings
        embeddings_index = get_GloVe_embeddings()
        # get embedding matrix for words in our vocab
        embedding_matrix = get_embedding_matrix(embeddings_index, word_index, num_words, embedding_dim, max_features)
        X_train, Y_train, sample_weights_train = create_sets(df_train, tokenizer, embedding_matrix)
        X_test, Y_test, sample_weights_test = create_sets(df_test, tokenizer, embedding_matrix)
        return X_train, Y_train, X_test, Y_test, sample_weights_train, sample_weights_test
    ####################################################################################################################


    embedding_dim = 200  # dim of our learned embedding model
    X_train, Y_train, X_test, Y_test, sample_weights_train, sample_weights_test = prep_data(df_train, df_test, embedding_dim)
    utterance_length = 40
    conversation_length = 20
    inputs = Input(shape=((conversation_length), utterance_length, embedding_dim,))

    # CNN Layer
    num_filters = 50
    reshape = Reshape((conversation_length, utterance_length, embedding_dim, 1))(inputs)
    conv_0_3 = TimeDistributed(Conv2D(num_filters, kernel_size=(2, embedding_dim), activation='relu'), input_shape=(1, conversation_length, utterance_length, embedding_dim, 1))(reshape)
    maxpool_0_3 = TimeDistributed(MaxPool2D(pool_size=(2, 1), padding='valid'))(conv_0_3)
    conv_1_3 = TimeDistributed(Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu'), input_shape=(1, conversation_length, utterance_length, embedding_dim, 1))(reshape)
    maxpool_1_3 = TimeDistributed(MaxPool2D(pool_size=(2, 1), padding='valid'))(conv_1_3)
    conv_2_3 = TimeDistributed(Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu'), input_shape=(1, conversation_length, utterance_length, embedding_dim, 1))(reshape)
    maxpool_2_3 = TimeDistributed(MaxPool2D(pool_size=(3, 1), padding='valid'))(conv_2_3)
    concatenated_tensor = Concatenate(axis=2)([maxpool_0_3, maxpool_1_3, maxpool_2_3])
    flatten = TimeDistributed(Flatten())(concatenated_tensor)
    output = Dropout(0.5)(flatten)

    # biLSTM Layer
    bilstm = Bidirectional(LSTM(units=200, return_sequences=True, recurrent_dropout=0.1))(output)  # variational biLSTM)
    outputs = TimeDistributed(Dense(1, activation="sigmoid"))(bilstm)
    model = Model(inputs=inputs, outputs=outputs)

    opt = adam(lr=0.001)
    model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"], sample_weight_mode='temporal')
    print(model.summary())

    ###################################################former_evaluation.py#############################################################
    import pandas as pd
    from sklearn.metrics import precision_recall_fscore_support
    from keras.callbacks import EarlyStopping
    import time
    from matplotlib import pyplot
    from numpy.random import seed
    from tensorflow import set_random_seed

    seed(1)
    set_random_seed(2)
    start_time = time.time()


    def write_csv(test_labels, pred_labels):
        for list in test_labels:
            list.insert(0, 'Question')
        for list in pred_labels:
            list.insert(0, 'Question')
        test_file = "../../data/answer/test_set.csv"
        test_df = pd.read_csv(test_file)
        grouped_by_conv = test_df.groupby(['thread'], sort=False)
        count = 0

        for thread, conv in grouped_by_conv:
            if conv.shape[0] != len(test_labels[count]):
                for i in range(conv.shape[0] - len(test_labels[count])):
                    test_labels[count].append(0)
                    pred_labels[count].append(0)
            count += 1

        test_list = [item for sublist in test_labels for item in sublist]
        pred_list = [item for sublist in pred_labels for item in sublist]
        test_df['TrueLabels'] = test_list
        test_df['PredLabels'] = pred_list
        test_df.to_csv("test_ChatE_pred.csv", index=False)


    def calc_eval_measures():
        print("Evaluation Measures:")
        df = pd.read_csv('./test_ChatE_pred.csv')
        true = df['TrueLabels'].values.tolist()
        true = [x for x in true if x != 'Question']  # discarding question instances
        true = ['1' if x == 'Answer' else x for x in true]  # converting to binary values
        true = ['0' if x == 'NonAnswer' else x for x in true]  # converting to binary values

        pred = df['PredLabels'].values.tolist()
        pred = [x for x in pred if x != 'Question']  # discarding question instances
        pred = ['1' if x == 'Answer' else x for x in pred]  # converting to binary values
        pred = ['0' if x == 'NonAnswer' else x for x in pred]  # converting to binary values

        precision, recall, fscore, support = precision_recall_fscore_support(true, pred, average='binary',
                                                                             pos_label='1')
        print('precision: {:.2f}'.format(precision))
        print('recall: {:.2f}'.format(recall))
        print('fscore: {:.2f}'.format(fscore))


    def pred2binarylabel(pred, sample_weights):
        idx2tag = {0: 'NonAnswer', 1: 'Answer'}
        out = []
        for i, pred_i in enumerate(pred):
            out_i = []
            for j, p in enumerate(pred_i):
                if p >= 0.5 and (sample_weights[i][j] == 1.0):
                    out_i.append(idx2tag[1])
                elif p < 0.5 and (sample_weights[i][j] == 1.0):
                    out_i.append(idx2tag[0])
            out.append(out_i)
        return out


    def train_test_validate(model, X_tr, Y_tr, X_te, Y_te, sample_weights_train, sample_weights_test):
        # Fit the model on train data
        early_stopping = EarlyStopping(monitor='val_loss', patience=10)
        history = model.fit(X_tr, Y_tr, batch_size=256, epochs=100, validation_split=0.2, callbacks=[early_stopping])
        pyplot.plot(history.history['loss'], label='train_loss')
        pyplot.plot(history.history['val_loss'], label='val_loss')
        pyplot.legend()
        pyplot.show()

        # Evaluate the model on test data
        scores = model.evaluate(X_te, Y_te, verbose=0)
        print("Accuracy: %.2f%%" % (scores[1] * 100))
        test_pred = model.predict(X_te, verbose=1)
        pred_labels = pred2binarylabel(test_pred, sample_weights_test)
        test_labels = pred2binarylabel(Y_te, sample_weights_test)
        print("Model took", time.time() - start_time, "to run")
        write_csv(test_labels, pred_labels)  # write predicted outputs
        calc_eval_measures()  # calculate evaluation measures
    ####################################################################################################################
    # Evaluation
    train_test_validate(model, X_train, Y_train, X_test, Y_test, sample_weights_train, sample_weights_test)


In [ ]:
Found 18273 unique tokens in vocab
Found 60000 word vectors.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
=================================================================
...
Total params: 3,245,601
Trainable params: 3,245,601
Non-trainable params: 0
_________________________________________________________________

In [ ]:
Evaluation Measures:
precision: 0.535
recall: 0.510
fscore: 0.500

Evaluation Measures:
precision: 0.462
recall: 0.421
fscore: 0.414
